In [3]:
import pandas as pd
import numpy as np


In [5]:
import pandas as pd

player = pd.read_csv(
    "C:/Users/caro_/Downloads/Proyecto Final DAPT10/player.csv"
)


In [6]:
team = pd.read_csv(
    "C:/Users/caro_/Downloads/Proyecto Final DAPT10/team.csv"
)

In [7]:
team_details = pd.read_csv(
    "C:/Users/caro_/Downloads/Proyecto Final DAPT10/team_details.csv"
)

In [8]:
team_history = pd.read_csv(
    "C:/Users/caro_/Downloads/Proyecto Final DAPT10/team_history.csv"
)

In [9]:
team_info_common = pd.read_csv(
    "C:/Users/caro_/Downloads/Proyecto Final DAPT10/team_info_common.csv"
)

In [4]:
play_by_play = pd.read_csv(
    "C:/Users/caro_/Downloads/Proyecto Final DAPT10/play_by_play.csv"
    )

In [10]:
# LIMPIEZA DE TABLA "PLAYER"
# Validar unicidad de id 
# Limpiar nombres:
# quitar espacios
# ---capitalización consistente
# Verificar coherencia:
#---full_name = first_name + last_name
# ---Convertir is_active a booleano
# ---Eliminar duplicados

player = player.drop_duplicates(subset="id")

player["first_name"] = player["first_name"].str.strip().str.title()
player["last_name"] = player["last_name"].str.strip().str.title()

player["full_name"] = (
    player["first_name"] + " " + player["last_name"]
)

player["is_active"] = player["is_active"].astype(bool)

player = player.rename(columns={"id": "player_id"})


In [11]:
# LIMPIEZA DE TABLA "TEAM"
#Renombrar id → team_id
# Normalizar texto (city, state, nickname)
#year_founded → entero
# Validar:

# año razonable (>=1900)
# Quitar duplicados


team = team.drop_duplicates(subset="id")

team = team.rename(columns={"id": "team_id"})

text_cols = ["full_name", "abbreviation", "nickname", "city", "state"]
for col in text_cols:
    team[col] = team[col].str.strip().str.title()

team["year_founded"] = pd.to_numeric(
    team["year_founded"], errors="coerce"
)

team = team[team["year_founded"] >= 1900]



In [12]:
# LIMPIEZA DE TABLA "TEAM_DETAILS"
# Unificar nombre de columna:
# yearfounded → year_founded
# arenacapacity → arena_capacity
# Validar:
# arena_capacity numérico
# Normalizar redes:
# lowercase
#quitar URLs incompletas
# Detectar columnas redundantes con archivo 2

team_details = team_details.rename(columns={
    "yearfounded": "year_founded",
    "arenacapacity": "arena_capacity"
})

team_details["arena_capacity"] = pd.to_numeric(
    team_details["arena_capacity"], errors="coerce"
)

text_cols = [
    "abbreviation", "nickname", "city", "arena",
    "owner", "generalmanager", "headcoach", "dleagueaffiliation"
]

for col in text_cols:
    team_details[col] = team_details[col].str.strip().str.title()

social_cols = ["facebook", "instagram", "twitter"]
for col in social_cols:
    team_details[col] = team_details[col].str.lower()
    team_details[col] = team_details[col].replace("", np.nan)


In [13]:
# # LIMPIEZA DE TABLA "TEAM_HISTORY"
# year_active_till:

#NULL → sigue activo
#Validar:

#year_active_till >= year_founded
# Detectar cambios de ciudad o nombre
# Ordenar cronológicamente

team_history["year_founded"] = pd.to_numeric(
    team_history["year_founded"], errors="coerce"
)

team_history["year_active_till"] = pd.to_numeric(
    team_history["year_active_till"], errors="coerce"
)

team_history = team_history[
    (team_history["year_active_till"].isna()) |
    (team_history["year_active_till"] >= team_history["year_founded"])
]

team_history["city"] = team_history["city"].str.strip().str.title()
team_history["nickname"] = team_history["nickname"].str.strip().str.title()


In [14]:
print(team_history.head()) 

      team_id        city    nickname  year_founded  year_active_till
0  1610612737     Atlanta       Hawks          1968              2019
1  1610612737   St. Louis       Hawks          1955              1967
2  1610612737   Milwaukee       Hawks          1951              1954
3  1610612737  Tri-Cities  Blackhawks          1949              1950
4  1610612741     Chicago       Bulls          1966              2019


In [16]:
# limpieza de tabla "PLAY_BY_PLAY"
# Validar:

# game_id no nulo
#  eventnum → entero
#  period → entero
#  Convertir tiempos:

# wctimestring

# pctimestring
#  Unificar descripciones:

# crear columna event_description
#  Eliminar eventos vacíos

play_by_play = play_by_play.dropna(subset=["game_id"])

int_cols = ["eventnum", "eventmsgtype", "eventmsgactiontype", "period"]
for col in int_cols:
    play_by_play[col] = pd.to_numeric(play_by_play[col], errors="coerce")

desc_cols = [
    "homedescription",
    "neutraldescription",
    "visitordescription"
]

for col in desc_cols:
    play_by_play[col] = play_by_play[col].fillna("")

play_by_play["event_description"] = (
    play_by_play["homedescription"] +
    play_by_play["neutraldescription"] +
    play_by_play["visitordescription"]
)

play_by_play = play_by_play.drop(columns=desc_cols)



In [17]:
print(play_by_play.head())

    game_id  eventnum  eventmsgtype  eventmsgactiontype  period wctimestring  \
0  29600012         0            12                   0       1     14:43 PM   
1  29600012         2            10                   0       1     14:50 PM   
2  29600012         3             2                   1       1     14:51 PM   
3  29600012         4             4                   0       1     14:51 PM   
4  29600012         5             2                   1       1     14:51 PM   

  pctimestring score scoremargin  person1type  ...  player2_team_abbreviation  \
0        12:00   NaN         NaN          0.0  ...                        NaN   
1        12:00   NaN         NaN          4.0  ...                        PHX   
2        11:45   NaN         NaN          5.0  ...                        NaN   
3        11:43   NaN         NaN          4.0  ...                        NaN   
4        11:29   NaN         NaN          4.0  ...                        NaN   

  person3type  player3_id player

In [18]:
# filtrar datos desde el año 2004 en la tabla "play_by_play"
game_info = pd.read_csv(
    "C:/Users/caro_/Downloads/Proyecto Final DAPT10/line_score_cleaned.csv"
    )


In [19]:
game_info["game_date_est"] = pd.to_datetime(
    game_info["game_date_est"],
    errors="coerce"
)


In [20]:
games_2004 = game_info[game_info["game_date_est"] >= "2004-01-01"]


In [23]:
game_info = game_info.merge(
    games_2004[["game_id", "game_date_est"]],
    on="game_id",
    how="inner"
)


In [24]:
# Guardar dataset limpio en nuevos archivos
output_path = "C:/Users\caro_/Downloads/Proyecto Final DAPT10"

player.to_csv(f"{output_path}player_clean.csv", index=False)
team.to_csv(f"{output_path}teams_core_clean.csv", index=False)
team_details.to_csv(f"{output_path}teams_details_clean.csv", index=False)
team_history.to_csv(f"{output_path}teams_history_clean.csv", index=False)

print("Archivo guardado exitosamente como 'teams_history_clean.csv'")


<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\caro_\AppData\Local\Temp\ipykernel_11384\4077846478.py:2: SyntaxWarning: invalid escape sequence '\c'
  output_path = "C:/Users\caro_/Downloads/Proyecto Final DAPT10"


Archivo guardado exitosamente como 'teams_history_clean.csv'


In [25]:
output_path = "C:/Users\caro_/Downloads/Proyecto Final DAPT10"

play_by_play.to_csv(f"{output_path}play_by_play_clean.csv", index=False)

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\caro_\AppData\Local\Temp\ipykernel_11384\1009662291.py:1: SyntaxWarning: invalid escape sequence '\c'
  output_path = "C:/Users\caro_/Downloads/Proyecto Final DAPT10"


In [26]:
print(play_by_play.shape)  

(13592899, 32)


In [27]:
(play_by_play.columns)

Index(['game_id', 'eventnum', 'eventmsgtype', 'eventmsgactiontype', 'period',
       'wctimestring', 'pctimestring', 'score', 'scoremargin', 'person1type',
       'player1_id', 'player1_name', 'player1_team_id', 'player1_team_city',
       'player1_team_nickname', 'player1_team_abbreviation', 'person2type',
       'player2_id', 'player2_name', 'player2_team_id', 'player2_team_city',
       'player2_team_nickname', 'player2_team_abbreviation', 'person3type',
       'player3_id', 'player3_name', 'player3_team_id', 'player3_team_city',
       'player3_team_nickname', 'player3_team_abbreviation',
       'video_available_flag', 'event_description'],
      dtype='object')

In [28]:
print(play_by_play.head()) 

    game_id  eventnum  eventmsgtype  eventmsgactiontype  period wctimestring  \
0  29600012         0            12                   0       1     14:43 PM   
1  29600012         2            10                   0       1     14:50 PM   
2  29600012         3             2                   1       1     14:51 PM   
3  29600012         4             4                   0       1     14:51 PM   
4  29600012         5             2                   1       1     14:51 PM   

  pctimestring score scoremargin  person1type  ...  player2_team_abbreviation  \
0        12:00   NaN         NaN          0.0  ...                        NaN   
1        12:00   NaN         NaN          4.0  ...                        PHX   
2        11:45   NaN         NaN          5.0  ...                        NaN   
3        11:43   NaN         NaN          4.0  ...                        NaN   
4        11:29   NaN         NaN          4.0  ...                        NaN   

  person3type  player3_id player